In [1]:
import os
import tempfile
from pathlib import Path
import pandas as pd
import numpy as np
import pyBigWig
import copy
import xarray as xr
import tqdm
import crandata


# Create temporary directories for synthetic data.
temp_dir = tempfile.TemporaryDirectory()
base_dir = Path(temp_dir.name)
beds_dir = base_dir / "beds"
bigwigs_dir = base_dir / "bigwigs"
beds_dir.mkdir(exist_ok=True)
bigwigs_dir.mkdir(exist_ok=True)

# Create a chromsizes file.
chromsizes_file = base_dir / "chrom.sizes"
with open(chromsizes_file, "w") as f:
    f.write("chr1\t1000\n")

# Create two BED files (simulate two different classes).
bed_data_A = pd.DataFrame({
    0: ["chr1", "chr1"],
    1: [100, 300],
    2: [200, 400]
})
bed_data_B = pd.DataFrame({
    0: ["chr1", "chr1"],
    1: [150, 350],
    2: [250, 450]
})
bed_file_A = beds_dir / "ClassA.bed"
bed_file_B = beds_dir / "ClassB.bed"
bed_data_A.to_csv(bed_file_A, sep="\t", header=False, index=False)
bed_data_B.to_csv(bed_file_B, sep="\t", header=False, index=False)

# Create a consensus BED file.
consensus = pd.DataFrame({
    0: ["chr1", "chr1", "chr1"],
    1: [100, 300, 350],
    2: [200, 400, 450]
})
consensus_file = base_dir / "consensus.bed"
consensus.to_csv(consensus_file, sep="\t", header=False, index=False)

# Create two bigWig files.
bigwig_file1 = bigwigs_dir / "test.bw"
bw1 = pyBigWig.open(str(bigwig_file1), "w")
bw1.addHeader([("chr1", 1000)])
bw1.addEntries(chroms=["chr1"], starts=[0], ends=[1000], values=[5.0])
bw1.close()

bigwig_file2 = bigwigs_dir / "test2.bw"
bw2 = pyBigWig.open(str(bigwig_file2), "w")
bw2.addHeader([("chr1", 1000)])
bw2.addEntries(chroms=["chr1"], starts=[0], ends=[1000], values=[4.0])
bw2.close()

# Set extraction parameters.
target_region_width = 100
backed_path = base_dir / "chrom_data.zarr"
print(backed_path)
# Create the CrAnData object from bigWig files and consensus regions.
adata = crandata.chrom_io.import_bigwigs(
    bigwigs_folder=str(bigwigs_dir),
    regions_file=str(consensus_file),
    backed_path=str(backed_path),
    target_region_width=target_region_width,
    chromsizes_file=str(chromsizes_file),
)

crandata.train_val_test_split(adata,strategy='chr_auto')

# Create a dummy FASTA file for a genome.
fasta_file = base_dir / "chr1.fa"
with open(fasta_file, "w") as f:
    f.write(">chr1\n")
    f.write("A" * 1000 + "\n")

dummy_genome = crandata.Genome(str(fasta_file), chrom_sizes=str(chromsizes_file))

# Add sequences to the CrAnData using the provided seq_io utility.
# Here we use the consensus regions as our ranges.
consensus.columns = ['chrom', 'start', 'end']
adata = crandata.seq_io.add_genome_sequences_to_crandata(adata, consensus, dummy_genome)

print(adata)
# Write the CrAnData object to disk and then reload it to ensure sequences are out-of-memory.
adata.to_zarr(str(backed_path),mode='a')
adata_loaded = crandata.CrAnData.open_zarr(str(backed_path))
print("Loaded CrAnData:")
print(adata_loaded)

# Create two copies to simulate two datasets (e.g. two species), and add a "split" column in var metadata.
adata1 = copy.deepcopy(adata_loaded)
adata2 = copy.deepcopy(adata_loaded)
adata1["var-_-split"] = xr.DataArray(np.full(adata1.sizes["var"], "train"), dims=["var"])
adata2["var-_-split"] = xr.DataArray(np.full(adata2.sizes["var"], "train"), dims=["var"])

# Create a DNATransform instance.
transform = crandata.seq_io.DNATransform(out_len=80, random_rc=True, max_shift=5)

# Instantiate the MetaCrAnDataModule with the two datasets.
# Note: The batch_size is now 3, matching the number of consensus regions (var dimension).
meta_module = crandata.MetaCrAnDataModule(
    adatas=[adata1, adata2],
    batch_size=[2,2],        # adjust batch size to not exceed var length (3)
    load_keys={'sequences':'sequences','X':'X'},
    shuffle=True,
    dnatransform=transform,
    epoch_size=10
)

meta_module.setup('train')

# Retrieve the training dataloader from the meta module and iterate over a couple of batches.
meta_train_dl = meta_module.train_dataloader
print("\nIterating over a couple of training batches from MetaCrAnDataModule:")
for i, batch in enumerate(tqdm.tqdm(meta_train_dl)):
    print(batch)
    print(f"\nMeta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
    if i >= 1:
        break

print("\nTemporary directory contents:")
print(os.listdir(base_dir))
temp_dir.cleanup()


/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/abc.py:106: FutureWarning: xarray subclass CrAnData should explicitly define __slots__
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)


/scratch/fast/168840/tmp99yl2au7/chrom_data.zarr


100%|██████████| 2/2 [00:00<00:00, 4362.25it/s]
2025-04-14 10:36:11.599 | INFO     | crandata.chrom_io:import_bigwigs:330 - Extracting values from 2 bigWig files...
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/core/array.py:3947: UserWarning: The dtype `StringDType()` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  result = await AsyncArray._create_v3(
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently no

<xarray.CrAnData> Size: 7kB
Dimensions:            (obs: 2, var: 3, seq_bins: 100, seq_len: 100, nuc: 4)
Coordinates:
  * obs                (obs) object 16B 'test' 'test2'
  * var                (var) object 24B 'chr1:100-200' ... 'chr1:350-450'
  * seq_bins           (seq_bins) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Dimensions without coordinates: seq_len, nuc
Data variables:
    obs-_-index        (obs) object 16B dask.array<chunksize=(2,), meta=np.ndarray>
    var-_-end          (var) int64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    obs-_-file_path    (obs) object 16B dask.array<chunksize=(2,), meta=np.ndarray>
    var-_-chunk_index  (var) int64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    var-_-chrom        (var) object 24B dask.array<chunksize=(3,), meta=np.ndarray>
    var-_-index        (var) object 24B dask.array<chunksize=(3,), meta=np.ndarray>
    var-_-start        (var) int64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    X                  (ob

0it [00:00, ?it/s]

{'sequences': array([[[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]]], dtype=uint8), 'X': array([[[6.90964373e-310, 6.90964373e-310, 4.68488233e-310,
         4.68488233e-310, 6.90943458e-310, 6.90943452e-310,
         6.90943482e-310, 4.68488233e-310, 6.90943501e-310,
         6.90960911e-310, 6.90943501e-310, 0.00000000e+000,
         6.90943617e-310, 6.90960911e-310, 6.90943617e-310,
         6.90943469e-310, 6.90960911e-310, 6.90943469e-310,
         6.90943469

1it [00:00, 18.01it/s]



Meta Batch 0:
  sequences: shape (4, 100, 4)
  X: shape (2, 4, 100)
{'sequences': array([[[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]]], dtype=uint8), 'X': array([[[6.90964373e-310, 6.90964373e-310, 4.68488233e-310,
         4.68488233e-310, 6.90943458e-310, 6.90943452e-310,
         6.90943482e-310, 4.68488233e-310, 6.90943501e-310,
         6.90960911e-310, 6.90943501e-310, 0.00000000e+000,
         6.90943617e-310, 6.90960911e-310, 6.90943617e-310,
         

In [2]:
sdfs

NameError: name 'sdfs' is not defined

In [ ]:
# Should the fill in _extract_values_from_bigwig actually be 0? Can we filter var where all is 0/nan without loading everything into memory?

In [1]:
import crandata
import xarray as xr
import pandas as pd
import numpy as np
import os
import crested
from tqdm import tqdm
import importlib

/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/abc.py:106: FutureWarning: xarray subclass CrAnData should explicitly define __slots__
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)


In [2]:
genomes = {}
beds = {}
chromsizes_files = {}
bed_files = {}
species = ['human','macaque','mouse']
species_codes = {'human':0,'macaque':1,'mouse':2}

MAX_SHIFT = 5
WINDOW_SIZE = 2114
WINDOW_SIZE = WINDOW_SIZE #+ 2*MAX_SHIFT
OFFSET = WINDOW_SIZE // 2  # e.g., 50% overlap
N_THRESHOLD = 0.3
n_bins = WINDOW_SIZE//50


In [3]:
for s in species:
    genome_path = '/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/genome/onehots/'+s
    fasta_file = os.path.join(genome_path,s+'.fa')
    chrom_sizes = os.path.join(genome_path,s+'.fa.sizes')
    annotation_gtf_file = os.path.join(genome_path,s+'.annotation.gtf')
    chromsizes_files[s] = chrom_sizes
    genome = crandata.Genome(fasta_file, chrom_sizes, annotation_gtf_file)
    genome.to_memory()
    genomes[s] = genome
    OUTPUT_BED = os.path.join(genome_path, "binned_genome.bed")
    bed_files[s] = OUTPUT_BED
    # Generate bins and optionally write to disk.
    binned_df = crandata.bin_genome(genome, WINDOW_SIZE, OFFSET, n_threshold=N_THRESHOLD, output_path=OUTPUT_BED).reset_index(drop=True)
    print("Filtered bins:")
    print(binned_df)


2025-04-15T00:05:12.810741-0700 INFO Genome sequences loaded into memory.


Calculating N content: 100%|██████████| 2932321/2932321 [04:46<00:00, 10250.21it/s]


Filtered bins:
              chrom  start    end   prop_n
0              chr1   9514  11628  0.23026
1              chr1  10571  12685  0.00000
2              chr1  11628  13742  0.00000
3              chr1  12685  14799  0.00000
4              chr1  13742  15856  0.00000
...             ...    ...    ...      ...
2786513  KI270518.1      1   2115  0.00000
2786514  KI270530.1      1   2115  0.00000
2786515  KI270304.1      1   2115  0.00000
2786516  KI270418.1      1   2115  0.00000
2786517  KI270424.1      1   2115  0.00000

[2786518 rows x 4 columns]
2025-04-15T00:10:51.943113-0700 INFO Genome sequences loaded into memory.


Calculating N content: 100%|██████████| 2806701/2806701 [04:31<00:00, 10348.06it/s]


Filtered bins:
               chrom  start    end  prop_n
0        NC_041754.1      1   2115     0.0
1        NC_041754.1   1058   3172     0.0
2        NC_041754.1   2115   4229     0.0
3        NC_041754.1   3172   5286     0.0
4        NC_041754.1   4229   6343     0.0
...              ...    ...    ...     ...
2773980  NC_005943.1   9514  11628     0.0
2773981  NC_005943.1  10571  12685     0.0
2773982  NC_005943.1  11628  13742     0.0
2773983  NC_005943.1  12685  14799     0.0
2773984  NC_005943.1  13742  15856     0.0

[2773985 rows x 4 columns]
2025-04-15T00:16:11.924751-0700 INFO Genome sequences loaded into memory.


Calculating N content: 100%|██████████| 2583507/2583507 [04:09<00:00, 10350.88it/s]


Filtered bins:
              chrom    start      end    prop_n
0              chr1  2999767  3001881  0.110638
1              chr1  3002938  3005052  0.085579
2              chr1  3003995  3006109  0.000000
3              chr1  3005052  3007166  0.000000
4              chr1  3006109  3008223  0.000000
...             ...      ...      ...       ...
2509462  JH584292.1     8457    10571  0.000000
2509463  JH584292.1     9514    11628  0.000000
2509464  JH584292.1    10571    12685  0.000000
2509465  JH584292.1    11628    13742  0.000000
2509466  JH584292.1    12685    14799  0.000000

[2509467 rows x 4 columns]


In [4]:
crandata.crandata = importlib.reload(crandata.crandata)
crandata.chrom_io = importlib.reload(crandata.chrom_io)

/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/abc.py:106: FutureWarning: xarray subclass CrAnData should explicitly define __slots__
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)


In [ ]:
adatas = {}

for s in species:
    print(s)
    bigwigs_dir = os.path.join('/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/SpinalCord/manuscript/ATAC',s,'Group_bigwig')
    adatas[s] = crandata.chrom_io.import_bigwigs(
        bigwigs_folder=bigwigs_dir,
        regions_file=bed_files[s],
        backed_path='/home/matthew.schmitz/Matthew/data/test_crandata/'+s+'_spc_test.zarr',
        target_region_width=WINDOW_SIZE,
        chromsizes_file=chromsizes_files[s],
        target = 'raw',
        max_stochastic_shift=5,
        tile_size=5000,
        chunk_size=512,
        n_bins=n_bins
    )
    bed = adatas[s].get_dataframe('var').loc[:,['chrom','start','end']]
    adatas[s] = crandata.seq_io.add_genome_sequences_to_crandata(adatas[s], bed, genomes[s])
    print(adatas[s]['sequences'])
    adatas[s]['var-_-species'] = xr.DataArray(np.repeat(species_codes[s],adatas[s].sizes['var']),dims='var').chunk({'var':adatas[s].attrs['chunk_size']})
    # adatas[s].to_icechunk(mode='a',commit_name='add_genome_seqs')
    adatas[s].to_zarr(adatas[s].encoding['source'],mode='a')
    # adatas[s] = crandata.crandata.CrAnData.open_zarr('/home/matthew.schmitz/Matthew/data/test_crandata/'+s+'_spc_test.zarr')
    

human


100%|██████████| 49/49 [00:01<00:00, 26.20it/s]


2025-04-15T00:21:17.125226-0700 INFO Extracting values from 49 bigWig files...


/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/code/grandada/crandata/chrom_io.py:386: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  adata.to_zarr(str(backed_path),mode='w')
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/core/array.py:3989: UserWarning: The dtype `StringDType()` is currently not part in the Zarr format 3 specification. It may not be sup

<xarray.DataArray 'sequences' (var: 2779994, seq_len: 2124, nuc: 4)> Size: 24GB
dask.array<xarray-<this-array>, shape=(2779994, 2124, 4), dtype=uint8, chunksize=(512, 2124, 4), chunktype=numpy.ndarray>
Coordinates:
  * var      (var) object 22MB 'chr1:9514-11628' ... 'KI270713.1:38053-40167'
Dimensions without coordinates: seq_len, nuc


...

/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/hom

mouse


100%|██████████| 49/49 [00:01<00:00, 29.92it/s]


2025-04-15T02:47:20.994900-0700 INFO Extracting values from 48 bigWig files...


/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/code/grandada/crandata/chrom_io.py:386: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  adata.to_zarr(str(backed_path),mode='w')
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/core/array.py:3989: UserWarning: The dtype `StringDType()` is currently not part in the Zarr format 3 specification. It may not be sup

In [ ]:
# #Alternate workflow to directly write icechunks, but this is ~5x slower (better to write pure zarr3 then convert the the whole store at once)
# adatas = {}

# for s in species:
#     print(s)
#     bigwigs_dir = os.path.join('/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/SpinalCord/manuscript/ATAC',s,'Group_bigwig')
#     adatas[s] = crandata.chrom_io.import_bigwigs(
#         bigwigs_folder=bigwigs_dir,
#         regions_file=bed_files[s],
#         backed_path='/home/matthew.schmitz/Matthew/data/test_crandata/'+s+'_spc_test.icechunk',
#         target_region_width=WINDOW_SIZE,
#         chromsizes_file=chromsizes_files[s],
#         target = 'raw',
#         max_stochastic_shift=5,
#         chunk_size=512,
#         backend='icechunk',
#         n_bins=n_bins
#     )
#     bed = adatas[s].get_dataframe('var').loc[:,['chrom','start','end']]
#     adatas[s] = crandata.seq_io.add_genome_sequences_to_crandata(adatas[s], bed, genomes[s])
#     print(adatas[s]['sequences'])
#     adatas[s]['var-_-species'] = xr.DataArray(np.repeat(species_codes[s],adatas[s].sizes['var']),dims='var').chunk({'var':adatas[s].attrs['chunk_size']})
#     adatas[s].to_icechunk(mode='a',commit_name='add_genome_seqs')
#     # adatas[s].to_zarr(adatas[s].encoding['source'],mode='a')
#     adatas[s] = crandata.crandata.CrAnData.open_icechunk('/home/matthew.schmitz/Matthew/data/test_crandata/'+s+'_spc_test.icechunk')
    

human


100%|██████████| 49/49 [00:00<00:00, 500.38it/s]


2025-04-14T12:02:42.163578-0700 INFO Extracting values from 49 bigWig files...


/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/core/array.py:3947: UserWarning: The dtype `StringDType()` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  result = await AsyncArray._create_v3(
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/

In [ ]:
#This inexplicably takes a long time...
for s in adatas.keys():
    crandata.train_val_test_split(
        adatas[s], strategy="region", val_size=0.1, test_size=0.1, random_state=42
    )
    adatas[s].to_zarr(adatas[s].encoding['source'],mode='a')
    # adatas[s].to_icechunk(mode='a',commit_name='train_val_test_split') #If you're using icechunk store

In [ ]:
print(adatas[s])
print(adatas[s]['X'])
adatas[s]['sequences']

In [3]:
adatas = {}

for s in tqdm(species):
    # adatas[s] = crandata.CrAnData.open_icechunk('/home/matthew.schmitz/Matthew/data/test_crandata/'+s+'_spc_test.icechunk',
    #                                             cache_config={'num_bytes_chunks':int(8e9)})#Cache 8Gb
    adatas[s] = crandata.crandata.CrAnData.open_zarr('/home/matthew.schmitz/Matthew/data/test_crandata/'+s+'_spc_test.zarr')

  0%|          | 0/3 [00:00<?, ?it/s]/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  re

In [25]:
crandata._module = importlib.reload(crandata._module)

In [26]:
transform = crandata.seq_io.DNATransform(out_len=WINDOW_SIZE, random_rc=True, max_shift=MAX_SHIFT)

meta_module = crandata._module.MetaCrAnDataModule(
    adatas=list(adatas.values()),
    batch_size=48,
    load_keys={'X': 'y','sequences':'sequence','var-_-species':'species'},
    dnatransform=transform,
    join='inner',
    num_workers=0,
    epoch_size=1000000
)

# Setup the meta module for the "fit" stage (train/val)
meta_module.setup("train")

# Retrieve the training dataloader from the meta module and iterate over a couple of batches.
meta_train_dl = meta_module.train_dataloader

print("\nIterating over a couple of training batches from MetaAnnDataModule:")
for i, batch in enumerate(tqdm(meta_train_dl)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
    if i == 5:
        break


/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/xbatcher/generators.py:112: UserWarning: The following dimensions were included in both ``input_dims`` and ``batch_dims``. Since ``concat_input_dims`` is ``False``, these dimensions will not impact batch generation: {'var': 16}
  warnings.warn(
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/xbatcher/generators.py:112: UserWarning: The following dimensions were included in both ``input_dims`` and ``batch_dims``. Since ``concat_input_dims`` is ``False``, these dimensions will not impact batch generation: {'var': 16}
  warnings.warn(
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/xbatcher/generators.py:112: UserWarning: The following dimensions were included in both ``input_dims`` and ``batch_dims``. Since ``concat_input_dims`` is ``False``, these dimensions will not impact batch generation: {'var': 16}
  warnings.warn


Iterating over a couple of training batches from MetaAnnDataModule:


1it [00:00,  1.25it/s]

Meta Batch 0:
  y: shape (47, 48, 42)
  sequence: shape (48, 2124, 4)
  species: shape (48,)


2it [00:01,  1.67it/s]

...

Meta Batch 3:
  y: shape (47, 48, 42)
  sequence: shape (48, 2124, 4)
  species: shape (48,)


5it [00:02,  2.17it/s]

Meta Batch 4:
  y: shape (47, 48, 42)
  sequence: shape (48, 2124, 4)
  species: shape (48,)


5it [00:02,  1.69it/s]

Meta Batch 5:
  y: shape (47, 48, 42)
  sequence: shape (48, 2124, 4)
  species: shape (48,)


In [27]:
import cProfile
import pstats

# Run your code and write the profile data to a file.
cProfile.run("""
for i, batch in tqdm(enumerate(meta_train_dl)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
    if i == 10:
        break
""", "profile_output.prof")

# Load the profile data from the file using pstats.
p = pstats.Stats("profile_output.prof")
p.strip_dirs().sort_stats("cumtime").print_stats(50)
# p.strip_dirs().sort_stats('cumtime').print_stats('crandata')


1it [00:00,  1.84it/s]

Meta Batch 0:
  y: shape (47, 48, 42)
  sequence: shape (48, 2124, 4)
  species: shape (48,)


2it [00:01,  1.97it/s]

Meta Batch 1:
  y: shape (47, 48, 42)
  sequence: shape (48, 2124, 4)
  species: shape (48,)


3it [00:01,  1.89it/s]

...

Meta Batch 8:
  y: shape (47, 48, 42)
  sequence: shape (48, 2124, 4)
  species: shape (48,)


10it [00:05,  1.84it/s]

Meta Batch 9:
  y: shape (47, 48, 42)
  sequence: shape (48, 2124, 4)
  species: shape (48,)


10it [00:05,  1.70it/s]

Meta Batch 10:
  y: shape (47, 48, 42)
  sequence: shape (48, 2124, 4)
  species: shape (48,)
Thu Apr 17 16:07:55 2025    profile_output.prof

         5672205 function calls (5639423 primitive calls) in 5.872 seconds

   Ordered by: cumulative time
   List reduced from 1392 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
4417/4393    0.015    0.000    6.838    0.002 {method 'run' of '_contextvars.Context' objects}
    16/12    0.000    0.000    5.867    0.489 std.py:1160(__iter__)
   104/11    0.001    0.000    5.865    0.533 base_node.py:75(__next__)
    12/11    0.000    0.000    5.865    0.533 loader.py:119(next)
  2013/17    0.002    0.000    5.860    0.345 {built-in method builtins.next}
    24/11    0.000    0.000    5.858    0.533 map.py:444(next)
    23/11    0.000    0.000    5.691    0.517 map.py:353(next)
    23/11    0.000    0.000    5.691    0.517 map.py:112(__next__)
       33    0.000    0.000    5.506    0.167 _mo

In [ ]:
for s in tqdm(species):
    adatas[s].unify_convert_chunks('/home/matthew.schmitz/Matthew/data/test_crandata/'+s+'_spc_test.icechunk')

In [49]:
adatas = {}
for s in tqdm(species):
    adatas[s] = crandata.CrAnData.open_icechunk('/home/matthew.schmitz/Matthew/data/test_crandata/'+s+'_spc_test.icechunk',
                                                cache_config={'num_bytes_chunks':int(8e9),'num_chunk_refs':5})#Cache 8Gb
    # adatas[s] = crandata.crandata.CrAnData.open_zarr('/home/matthew.schmitz/Matthew/data/test_crandata/'+s+'_spc_test.zarr')

  0%|          | 0/3 [00:00<?, ?it/s]/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  re

In [ ]:
transform = crandata.seq_io.DNATransform(out_len=WINDOW_SIZE, random_rc=True, max_shift=MAX_SHIFT)

meta_module = crandata.MetaCrAnDataModule(
    adatas=list(adatas.values()),
    batch_size=[16,16,16],
    load_keys={'X': 'y','sequences':'sequence','var-_-species':'species'},
    dnatransform=transform,
    join='inner',
    num_workers=0,
    epoch_size=1000000
)

# Setup the meta module for the "fit" stage (train/val)
meta_module.setup("train")

# Retrieve the training dataloader from the meta module and iterate over a couple of batches.
meta_train_dl = meta_module.train_dataloader

print("\nIterating over a couple of training batches from MetaAnnDataModule:")
for i, batch in enumerate(tqdm(meta_train_dl)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
    if i == 5:
        break


In [ ]:
import cProfile

code = '''
for i, batch in enumerate(tqdm(meta_train_dl)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
    if i == 5:
        break
'''

out = cProfile.run(code,sort=True)


In [22]:
meta_module.load()
meta_train_dl = meta_module.train_dataloader



KeyboardInterrupt



In [ ]:
code = '''
for i, batch in enumerate(tqdm(meta_train_dl)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
    if i == 50:
        break
'''

out = cProfile.run(code,sort=True)


In [ ]:
model_architecture = crested.tl.zoo.simple_convnet(
    seq_len=2114, num_classes=batch['y'].shape[1]
)


In [ ]:
import keras
# Create your own configuration
# I recommend trying this for peak regression with a weighted cosine mse log loss function
optimizer = keras.optimizers.Adam(learning_rate=1e-5)
loss = crested.tl.losses.CosineMSELogLoss(max_weight=100, multiplier=1)
loss = crested.tl.losses.PoissonLoss()

metrics = [
    keras.metrics.MeanAbsoluteError(),
    # keras.metrics.MeanSquaredError(),
    # keras.metrics.CosineSimilarity(axis=1),
    crested.tl.metrics.PearsonCorrelation(),
    # crested.tl.metrics.ConcordanceCorrelationCoefficient(),
    # crested.tl.metrics.PearsonCorrelationLog(),
    # crested.tl.metrics.ZeroPenaltyMetric(),
]

alternative_config = crested.tl.TaskConfig(optimizer, loss, metrics)
print(alternative_config)


In [ ]:
batch['sequence'].shape

In [ ]:
# initialize some lazy model parameters *yawn*
model_architecture(batch['sequence'].float().mean(0).unsqueeze(0))

In [ ]:
trainer = crested.tl.Crested(
    data=meta_module,
    model=model_architecture,
    config=alternative_config,
    project_name="mouse_biccn",  # change to your liking
    run_name="basemodel",  # change to your liking
    logger=None,  # or None, 'dvc', 'tensorboard'
    seed=7,  # For reproducibility
)
# train the model
trainer.fit(
    epochs=60,
    learning_rate_reduce_patience=3,
    early_stopping_patience=6,
)


In [95]:
import icechunk as ic
store_path = '/home/matthew.schmitz/Matthew/data/test_crandata/mouse_spc_test.icechunk'
storage_config = ic.local_filesystem_storage(store_path)
config = ic.RepositoryConfig.default()
config.caching = ic.CachingConfig(num_bytes_chunks=int(8e9))
repo = ic.Repository.open(storage_config, config)
session = repo.readonly_session("main")
ds = xr.open_zarr(session.store, consolidated=False)


/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/crested/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)


In [59]:
for k in ds.keys():
    print(k)
    print(ds[k].chunks)


X
((48,), (512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,

In [99]:
start_idx = np.random.randint(0, ds.dims['var'] - 100)
print("Random access start index:", start_idx)

t0 = time.time()
subset1 = ds.isel(var=slice(start_idx, start_idx + 100))['X'].values
print("First access time: {:.4f} sec".format(time.time() - t0))

t0 = time.time()
subset2 = ds.isel(var=slice(start_idx, start_idx + 100))['X'].values
print("Second access time: {:.4f} sec".format(time.time() - t0))

print("Done.")


Random access start index: 1822385
First access time: 0.0130 sec
Second access time: 0.0102 sec
Done.


/scratch/fast/198623/ipykernel_3971688/2594299141.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  start_idx = np.random.randint(0, ds.dims['var'] - 100)


In [94]:
start_idx = np.random.randint(0, ds.dims['var'] - 100)
print("Random access start index:", start_idx)

t0 = time.time()
subset1 = np.array(ds.isel(var=slice(start_idx, start_idx + 100))['X'])
print("First access time: {:.4f} sec".format(time.time() - t0))

t0 = time.time()
subset2 = np.array(ds.isel(var=slice(start_idx, start_idx + 100))['X'].values)
print("Second access time: {:.4f} sec".format(time.time() - t0))

print("Done.")


Random access start index: 686658
First access time: 0.0125 sec
Second access time: 0.0112 sec
Done.


/scratch/fast/198623/ipykernel_3971688/481843873.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  start_idx = np.random.randint(0, ds.dims['var'] - 100)


In [61]:
%%time
np.array(ds.isel({'var':np.arange(start,start+1000)})['X'])
print('done')

<timed exec>:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)


done
CPU times: user 323 ms, sys: 123 ms, total: 446 ms
Wall time: 442 ms


In [62]:
%%time
np.array(ds.isel({'var':np.arange(start,start+1000)})['X'])
print('done')

<timed exec>:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)


done
CPU times: user 316 ms, sys: 114 ms, total: 430 ms
Wall time: 423 ms


In [63]:
ds.isel({'var':np.arange(start,start+1000)})['X']

/scratch/fast/198623/ipykernel_3971688/567680973.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ds.isel({'var':np.arange(start,start+1000)})['X']


<xarray.DataArray 'X' (obs: 48, var: 1000, seq_bins: 42)> Size: 16MB
dask.array<getitem, shape=(48, 1000, 42), dtype=float64, chunksize=(48, 511, 42), chunktype=numpy.ndarray>
Coordinates:
  * obs       (obs) object 384B 'Alpha-MN' 'CSF-cNs' ... 'WM_Astrocytes'
  * seq_bins  (seq_bins) int64 336B 0 1 2 3 4 5 6 7 ... 34 35 36 37 38 39 40 41
  * var       (var) object 8kB 'chr8:76524687-76526801' ... 'chr8:77580630-77...

In [115]:
import tempfile
import time
import numpy as np
import xarray as xr
import icechunk as ic
from icechunk.xarray import to_icechunk

# Create a temporary directory for the Icechunk store
with tempfile.TemporaryDirectory() as tmpdir:
    store_path = f"{tmpdir}/example.icechunk"
    
    # Set up local storage and repository with a caching configuration (1 MB in this demo)
    storage_config = ic.local_filesystem_storage(store_path)
    config = ic.RepositoryConfig.default()
    config.caching = ic.CachingConfig(num_bytes_chunks=1024 * 1024)
    repo = ic.Repository.create(storage_config, config)
    
    # Create a simple xarray dataset with a 'var' dimension and chunk it along 'var'
    # In this example, the dataset 'X' has shape (var=1000, y=20) and chunks of size 100 along 'var'
    data = np.random.rand(100000, 20)
    ds = xr.Dataset({'X': (('var', 'y'), data)})
    ds = ds.chunk({'var': 100, 'y': 20})
    
    # Write the dataset to the Icechunk store using a writable session
    session = repo.writable_session("main")
    to_icechunk(ds, session)
    commit_hash = session.commit("initial commit")
    print("Committed with hash:", commit_hash)
    
    # Read the dataset back from the store using a read-only session
    session = repo.readonly_session("main")
    ds2 = xr.open_zarr(session.store, consolidated=False)
    print("Dataset dimensions:", ds2.dims)
    
    # Test the caching behavior by timing two consecutive random accesses along 'var'
    var_dim = ds2.dims['var']
    # Ensure we have 100 contiguous indices available (avoid overflow)
    start_idx = np.random.randint(0, var_dim - 100)
    print("Random access start index:", start_idx)
    
    t0 = time.time()
    subset1 = ds2.isel(var=slice(start_idx, start_idx + 100))['X'].values
    print("First access time: {:.4f} sec".format(time.time() - t0))
    
    t0 = time.time()
    subset2 = ds2.isel(var=slice(start_idx, start_idx + 100))['X'].values
    print("Second access time: {:.4f} sec".format(time.time() - t0))
    
    print("Done.")


Committed with hash: V6J86ASCJMYC72RA9NR0
Dataset dimensions: FrozenMappingWarningOnValuesAccess({'var': 100000, 'y': 20})
Random access start index: 49587
First access time: 0.0063 sec
Second access time: 0.0047 sec
Done.


/scratch/fast/198623/ipykernel_3971688/3493762518.py:36: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  var_dim = ds2.dims['var']


In [66]:
1024 * 1024

1048576